In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
import wide_residual_network as wrn
import keras.callbacks as callbacks

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [ ]:
def schedule(x):
    if x < 60: 
        return 0.1
    elif x < 120:
        return 0.1*0.2
    elif x < 160:
        return 0.1*0.2*0.2
    else: 
        return 0.1*0.2*0.2*0.2

In [ ]:
# Callbacks for the model
logs_dir = ""
lr_1 = callbacks.LearningRateScheduler(schedule)
csv_logger = callbacks.CSVLogger(logs_dir+'csv_train_logs.csv')

In [3]:
act, act_name = e_swish_2, "e_swish_2"

In [4]:
# data augmentation - no rotation range as described in the paper
datagen = ImageDataGenerator(
#     rotation_range=15,
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True, 
    fill_mode="reflect"
    )
datagen.fit(x_train)

In [ ]:
init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)

# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
model = wrn.create_wide_residual_network(act, init_shape, nb_classes=10, N=2, k=4, dropout=0.00)

model.summary()

Wide Residual Network-10-4 created.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_________________________________________________________________________

In [ ]:
batch_size  = 128
epochs = 200

opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=5e-4, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Training up to 200 epochs")

his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1, csv_logger])
model.save('wrn_16_4_00.h5')
print(his.history)

Finished compiling
Training up to 60 epochs
Epoch 1/30
781/781 [==============================] - 1039s 1s/step - loss: 1.4287 - acc: 0.4728 - val_loss: 1.0929 - val_acc: 0.6047
Epoch 2/30
781/781 [==============================] - 1014s 1s/step - loss: 0.9793 - acc: 0.6506 - val_loss: 0.8898 - val_acc: 0.6855
Epoch 3/30
781/781 [==============================] - 1014s 1s/step - loss: 0.8046 - acc: 0.7157 - val_loss: 0.7593 - val_acc: 0.7342
Epoch 4/30
781/781 [==============================] - 1020s 1s/step - loss: 0.6903 - acc: 0.7568 - val_loss: 0.6518 - val_acc: 0.7678
Epoch 5/30
781/781 [==============================] - 1017s 1s/step - loss: 0.6069 - acc: 0.7877 - val_loss: 0.5676 - val_acc: 0.8067
Epoch 6/30
781/781 [==============================] - 1014s 1s/step - loss: 0.5479 - acc: 0.8081 - val_loss: 0.5255 - val_acc: 0.8183
Epoch 7/30
781/781 [==============================] - 1014s 1s/step - loss: 0.4934 - acc: 0.8266 - val_loss: 0.4999 - val_acc: 0.8274
Epoch 8/30
781/781

KeyboardInterrupt: 